In [1]:
from superpixels import export_superpixel_crf_masks_for_dataset
from dataset import Colonoscopy_Dataset
from tools import (
    return_files_in_directory,
    human_sort,
    overlay_mask_on_image
    )
from crf import pass_pseudomask_or_ground_truth
from crf import crf
from config import *
import os
from embeddings import ResnetFeatureExtractor, generate_embedding_masks_for_dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
import torchvision
from tqdm import tqdm
from tools import visualize
from embeddings import get_mean_embeddings
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries, slic
import numpy as np

import time
from embeddings import create_embedding_mask
from crf import pass_pseudomask_or_ground_truth
from embeddings import ResnetFeatureExtractor

/Users/michaelgroeger/miniconda3/envs/boxshrink/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
image_files = return_files_in_directory(DATA_DIR + "/original", ".tif")
box_files = return_files_in_directory(DATA_DIR + "/boxmasks", ".png")
# Ensure files are in correct order
human_sort(image_files)
human_sort(box_files)
X_train, X_test, y_train, y_test = train_test_split(image_files, box_files, test_size=0.1, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.11111, random_state=1) # 0.1111 x 0.9 = 0.1

dataset = Colonoscopy_Dataset(X_train[:5], y_train[:5])
data_loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=0)

TESTING_DIR = DATA_DIR + "/testing/robust_boxshrink"
EMBEDDING_DIR = DATA_DIR + "/testing/mean_embeddings/"
if not os.path.exists(TESTING_DIR):
    os.makedirs(TESTING_DIR)

if not os.path.exists(EMBEDDING_DIR):
    os.makedirs(EMBEDDING_DIR)

In [5]:
## Get mean embeddings
f = torch.load(EMBEDDING_DIR + "foreground_embeddings_ns250_th01_sp_crf_masks.pt")
b = torch.load(EMBEDDING_DIR + "background_embeddings_ns250_th01_sp_crf_masks.pt")
mean_f = torch.mean(f, dim=0)
mean_b = torch.mean(b, dim=0)

In [7]:
resnet = torchvision.models.resnet50(weights="ResNet50_Weights.IMAGENET1K_V2")
resnet.eval()
feature_extract_model = ResnetFeatureExtractor(resnet)

feature_extract_model.to(DEVICE)
res = generate_embedding_masks_for_dataset(dataset, '/Users/michaelgroeger/workspace/FEA_Internship/publication/boxshrink/tests/test_embedding_masks', feature_extract_model, mean_f, mean_b, save_as_png=False)

Generating Embedding Masks:   0%|          | 0/8 [00:35<?, ?batch/s]


KeyboardInterrupt: 